<a href="https://colab.research.google.com/github/Kaleab1999/Ham-Spam-classifier/blob/main/uni_gram_with_bow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## spam checker

importing pandas to load our dataset

In [ ]:
import pandas as pd

1. loading the dataset
2. separate the dataset by 't' or tab
3. label the separatted data 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/MyDrive/dataset/ssms_spam.txt',sep='\t',
                           names=["label", "message"])
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


## Data cleaning

In [ ]:
import re #for lowerring text, and maintaining words
import nltk #for NLP (to perform tokenization,stop_words,stemming,lemittization,bag_of_words,TFIDF)

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords # to apply stopwords
from nltk.stem.porter import PorterStemmer #for stemming
from nltk.stem import WordNetLemmatizer #to perform Lemmatizzation

creating object for stemming and lemmatization

In [ ]:
ps = PorterStemmer()
corpus = []
lemmatize=WordNetLemmatizer()
corpus2=[]

In [ ]:
for i in range(0, len(df)):
    review2 = re.sub('[^a-zA-Z]', ' ', df['message'][i])#to remove all except letters
    review2 = review2.lower() #lowerring the letter
    review2 = review2.split() #tokenize
    
    review2 = [ps.stem(word) for word in review2 if not word in stopwords.words('english')]
    review2 = ' '.join(review2)
    corpus.append(review2)

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
for i in range(0, len(df)):
    review = re.sub('[^a-zA-Z]', ' ', df['message'][i])#to remove all except letters
    review = review.lower() #to change the word into lowercase
    review = review.split()#word tokenization
    
    review = [lemmatize.lemmatize(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus2.append(review)

In [ ]:
import textblob
from textblob import TextBlob

In [ ]:
df['polarity']=df['message'].apply(lambda tweet:TextBlob(tweet).sentiment.polarity)
df['subjectivity']=df['message'].apply(lambda tweet:TextBlob(tweet).sentiment.subjectivity)

df.drop(['message'], axis=1, inplace=True)
df.head()


,label,polarity,subjectivity
0,ham,0.15,0.7625
1,ham,0.50,0.5000
2,spam,0.30,0.5500
3,ham,0.10,0.3000
4,ham,0.00,0.0000


### Creating the Bag of Words model

In [ ]:

from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=2500)
X = cv.fit_transform(corpus2).toarray()

y=pd.get_dummies(df['label'])
y=y.iloc[:,1].values

In [ ]:
y= pd.DataFrame(y)
y.head()

,0
0,0
1,0
2,1
3,0
4,0


In [ ]:
x=pd.DataFrame(X)
k=df[['polarity','subjectivity']]
p=pd.DataFrame(k)
X=pd.concat([x,p],axis=1)

X.head()

,0,1,2,3,4,5,6,7,8,9,...,2494,2495,2496,2497,2498,2499,polarity,subjectivity,polarity,subjectivity
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.15,0.7625,0.15,0.7625
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.50,0.5000,0.50,0.5000
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.30,0.5500,0.30,0.5500
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.10,0.3000,0.10,0.3000
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.00,0.0000,0.00,0.0000


splitting the dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [ ]:
# Training model 

from sklearn.ensemble import RandomForestClassifier

randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(criterion='entropy', n_estimators=200)

In [ ]:
## Predict for the Test Dataset

predictions = randomclassifier.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
y_test.columns

RangeIndex(start=0, stop=1, step=1)

In [ ]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],predictions)
print(matrix)
score=accuracy_score(y_test[0],predictions)
print(score)
report=classification_report(y_test[0],predictions)
print(report)

[[966   0]
 [ 21 128]]
0.9811659192825112
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.86      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [ ]:
from sklearn.linear_model import  LogisticRegression

logreg=LogisticRegression(C=1e5, max_iter=10000 )
logreg.fit(X_train, y_train)

pred = logreg.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],pred)
print(matrix)
score=accuracy_score(y_test[0],pred)
print(score)
report=classification_report(y_test[0],pred)
print(report)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[[963   3]
 [ 12 137]]
0.9865470852017937
              precision    recall  f1-score   support

           0       0.99      1.00      0.99       966
           1       0.98      0.92      0.95       149

    accuracy                           0.99      1115
   macro avg       0.98      0.96      0.97      1115
weighted avg       0.99      0.99      0.99      1115



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
randomclassifier=RandomForestClassifier(n_estimators=200)
randomclassifier.fit(X_train,y_train)

pred= randomclassifier.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],pred)
print(matrix)
score=accuracy_score(y_test[0],pred)
print(score)
report=classification_report(y_test[0],pred)
print(report)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[[966   0]
 [ 17 132]]
0.9847533632286996
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.89      0.94       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.97      1115
weighted avg       0.99      0.98      0.98      1115



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

pred= gnb.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],pred)
print(matrix)
score=accuracy_score(y_test[0],pred)
print(score)
report=classification_report(y_test[0],pred)
print(report)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


[[835 131]
 [ 15 134]]
0.8690582959641255
              precision    recall  f1-score   support

           0       0.98      0.86      0.92       966
           1       0.51      0.90      0.65       149

    accuracy                           0.87      1115
   macro avg       0.74      0.88      0.78      1115
weighted avg       0.92      0.87      0.88      1115



In [ ]:
import numpy as np
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt','log2']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 1000,10)]
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10,14]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4,6,8]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
              'criterion':['entropy','gini']}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt', 'log2'], 'max_depth': [10, 120, 230, 340, 450, 560, 670, 780, 890, 1000], 'min_samples_split': [2, 5, 10, 14], 'min_samples_leaf': [1, 2, 4, 6, 8], 'criterion': ['entropy', 'gini']}


In [ ]:
rf=RandomForestClassifier()
rf_randomcv=RandomizedSearchCV(estimator=rf,param_distributions=random_grid,n_iter=100,cv=3,verbose=2,
                               random_state=100,n_jobs=-1)
rf_randomcv.fit(X_train,y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:705: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:926: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [10, 120, 230, 340, 450,
                                                      560, 670, 780, 890,
                                                      1000],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6, 8],
                                        'min_samples_split': [2, 5, 10, 14],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=100, verbose=2)

In [ ]:
rf_randomcv.best_params_

{'criterion': 'entropy',
 'max_depth': 450,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 14,
 'n_estimators': 200}

In [ ]:

randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy',max_depth= 450,max_features= 'sqrt' , min_samples_split=14, min_samples_leaf=1)
randomclassifier.fit(X_train,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(criterion='entropy', max_depth=450, max_features='sqrt',
                       min_samples_split=14, n_estimators=200)

In [ ]:
pred= randomclassifier.predict(X_test)

from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

matrix=confusion_matrix(y_test[0],pred)
print(matrix)
score=accuracy_score(y_test[0],pred)
print(score)
report=classification_report(y_test[0],pred)
print(report)

[[966   0]
 [ 18 131]]
0.9838565022421525
              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.88      0.94       149

    accuracy                           0.98      1115
   macro avg       0.99      0.94      0.96      1115
weighted avg       0.98      0.98      0.98      1115



/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
